In [122]:
import polars as pl
import re
from unidecode import unidecode
from datetime import datetime, timedelta

## Fonctions :

In [123]:
def date_serial_number(serial_number: int) -> datetime:
    """
    Convert an Excel serial number to a Python datetime object
    :param serial_number: the date serial number
    :return: a datetime object
    """
    # Excel stores dates as "number of days since 1900"

    delta = datetime(1899, 12, 30) + timedelta(days=serial_number)
    return delta

In [124]:
def standardize_column_name(df: pl.DataFrame) -> pl.DataFrame:
    """
    Define a function to apply regex and transformations on column names
    """
    def clean_name(col: str) -> str:
        col = re.sub(r'(?<!^)(?=[A-Z])|(?<=_)(?=[A-Z])| |-', lambda x: "_" if x.group(0) == " " else "", unidecode(col.lower()))
        col = col.replace("'", "_").replace("__", "_")
        return col
    
    return df.rename({col: clean_name(col) for col in df.columns})

In [125]:
def get_infos(df: pl.DataFrame, file=None, head=True, display_memory=False):
    """
    Provides basic information about a Polars DataFrame including shape, duplicates, 
    and missing values.

    Parameters:
    -----------
    df : pl.DataFrame
        The Polars DataFrame to be analyzed.
    
    file : str, optional
        The name of the file or source of the DataFrame. Default is None.
        If provided, it will be displayed in the output.
    
    head : bool, optional
        A flag indicating whether to display the first few rows of the DataFrame.
        Default is True. If set to True, `df.head()` will be printed.

    Returns:
    --------
    None
        The function prints out information about the DataFrame such as the number of 
        rows, columns, duplicated rows, percentage of missing values, and optionally 
        the first few rows of the DataFrame.

    Notes:
    ------
    - The function computes the percentage of missing values based on the sum of 
      all missing values across all columns.
    - Duplicated rows are identified using Polars' `is_duplicated()` method.
    """

    if df.shape[0] == 0 or df.shape[1] == 0:
        print("The DataFrame is empty.")
        return
    
    # Count duplicated rows
    duplicated_rows = df.is_duplicated().sum()
    
    # Calculate the total number of missing values
    isnull_total = df.null_count().sum_horizontal()
    
    # Calculate the total number of values (rows * columns)
    value_total = df.shape[0] * df.shape[1]
    
    # Compute the percentage of missing values
    pct_missing = isnull_total / value_total

    memory_usage_mb = (df.shape[0] * df.shape[1] * 8) / (1024 ** 2)

    # If a file name is provided, print it
    if file:
        print(f"Nom du fichier : {file}")
    
    # Display basic shape information
    print(f"Nombre de lignes : {df.shape[0]}")
    print(f"Nombre de colonnes : {df.shape[1]}")
    print("------")
    
    # Display duplicated rows and missing values percentage
    print(f"Nombre de lignes en double : {duplicated_rows}")
    print(f"Pourcentage de valeurs manquantes : {pct_missing.item():.2%}")
   
    if display_memory == True:
    # Display memory usage
        print(f"Estimated memory usage: {memory_usage_mb:.2f} MB")
    
    # Optionally print the first few rows of the DataFrame
    if head:
        print(f"-----")
        print(df.head())

## Variables

In [126]:
FILE_PATH = 'documents/extraction_olap.xlsx'
SHEET_1 = 'Vente Détail'
SHEET_2 = 'Produits'
SHEET_3 = 'Clients'
SHEET_4 = 'Calendrier'
SHEET_5 = 'Employé'

## Importations

In [127]:
df_vente = pl.read_excel(FILE_PATH, sheet_name=SHEET_1)

df_prod = pl.read_excel(FILE_PATH, sheet_name=SHEET_2)

df_client = pl.read_excel(FILE_PATH, sheet_name=SHEET_3)

df_calendrier = pl.read_excel(FILE_PATH, sheet_name=SHEET_4)

df_employe = pl.read_excel(FILE_PATH, sheet_name=SHEET_5)

In [128]:
df_vente = standardize_column_name(df_vente)

df_prod = standardize_column_name(df_prod)

df_client = standardize_column_name(df_client)

df_calendrier = standardize_column_name(df_calendrier)

df_employe = standardize_column_name(df_employe)

## Exploration

### 1. Table vente

In [129]:
get_infos(df_vente)

Nombre de lignes : 41377
Nombre de colonnes : 6
------
Nombre de lignes en double : 0
Pourcentage de valeurs manquantes : 0.00%
-----
shape: (5, 6)
┌───────────────────┬──────────────────┬──────────────────┬───────────────┬────────────┬───────────┐
│ id_bdd            ┆ customer_id      ┆ id_employe       ┆ ean           ┆ date_achat ┆ id_ticket │
│ ---               ┆ ---              ┆ ---              ┆ ---           ┆ ---        ┆ ---       │
│ str               ┆ str              ┆ str              ┆ i64           ┆ i64        ┆ str       │
╞═══════════════════╪══════════════════╪══════════════════╪═══════════════╪════════════╪═══════════╡
│ HZDG8U15NNY7SI6HD ┆ CUST-G42Z6WE8QLW ┆ b413ca065a762e8c ┆ 6473630445822 ┆ 45518      ┆ t_2693    │
│ K8NMFDEK7MOVU…    ┆ J                ┆ f2e86cfea8b9c1…  ┆               ┆            ┆           │
│ 1H51BRR800TK9DCIH ┆ CUST-CUA37GP8GAB ┆ a7ada0770091e838 ┆ 1857802002765 ┆ 45518      ┆ t_4408    │
│ 8M9QCRH3LEAR0…    ┆ Q                ┆ e3d

In [130]:
df_vente = (
    df_vente.with_columns(
        pl.col('date_achat')
        .map_elements(lambda x: date_serial_number(int(x)).strftime("%Y-%m-%d"), return_dtype=pl.Utf8)
        .alias('date_achat')
    )
)

In [131]:
df_vente.select('date_achat').n_unique()

1

In [132]:
df_vente.select([pl.col(column).n_unique() for column in df_vente.columns])

id_bdd,customer_id,id_employe,ean,date_achat,id_ticket
u32,u32,u32,u32,u32,u32
41377,2297,56,16146,1,1808


### 2. Table produit

In [133]:
get_infos(df_prod)

Nombre de lignes : 18040
Nombre de colonnes : 5
------
Nombre de lignes en double : 0
Pourcentage de valeurs manquantes : 0.00%
-----
shape: (5, 5)
┌───────────────┬───────────────────────────┬───────┬───────────────────────────┬──────┐
│ ean           ┆ categorie                 ┆ rayon ┆ libelle_produit           ┆ prix │
│ ---           ┆ ---                       ┆ ---   ┆ ---                       ┆ ---  │
│ i64           ┆ str                       ┆ str   ┆ str                       ┆ f64  │
╞═══════════════╪═══════════════════════════╪═══════╪═══════════════════════════╪══════╡
│ 5026767366043 ┆ Produits Secs & Conserves ┆ pates ┆ 500g penne rigate panzani ┆ 1.24 │
│ 1002603715237 ┆ Produits Secs & Conserves ┆ pates ┆ 500g spaghetti panzani    ┆ 0.84 │
│ 2113941413715 ┆ Produits Secs & Conserves ┆ pates ┆ 1kg coquillettes panzani  ┆ 1.73 │
│ 2597945667827 ┆ Produits Secs & Conserves ┆ pates ┆ 1kg spaghetti panzani     ┆ 1.53 │
│ 8046456922921 ┆ Produits Secs & Conserves ┆ pates

In [134]:
df_prod.select([pl.col(column).n_unique() for column in df_prod.columns])

ean,categorie,rayon,libelle_produit,prix
u32,u32,u32,u32,u32
18040,15,128,17972,1789


In [135]:
print(df_prod['categorie'].value_counts(sort=True))

shape: (15, 2)
┌──────────────────────────────┬───────┐
│ categorie                    ┆ count │
│ ---                          ┆ ---   │
│ str                          ┆ u32   │
╞══════════════════════════════╪═══════╡
│ Produits Secs & Conserves    ┆ 4566  │
│ Hygiène & Parfumerie         ┆ 3322  │
│ Cosmétiques & Maquillage     ┆ 1756  │
│ Boissons                     ┆ 1431  │
│ Produits Laitiers & Crèmerie ┆ 1342  │
│ …                            ┆ …     │
│ Soins de la Maison           ┆ 400   │
│ Soins & Produits Bébé        ┆ 337   │
│ Pâtisseries Emballées        ┆ 289   │
│ Maison & Décoration          ┆ 231   │
│ Produits de Volaille         ┆ 130   │
└──────────────────────────────┴───────┘


In [136]:
df_prod.select('categorie').unique().to_series().to_list()

['Bricolage & Outillage',
 'Boulangerie & Viennoiseries',
 'Pâtisseries Emballées',
 'Hygiène & Parfumerie',
 'Produits Laitiers & Crèmerie',
 'Soins & Produits Bébé',
 'Produits Secs & Conserves',
 'Jouets & Jeux',
 'Charcuterie & Plats Traiteur',
 'Soins de la Maison',
 'Maison & Décoration',
 'Produits de Volaille',
 'Produits Surgelés & Préparés',
 'Boissons',
 'Cosmétiques & Maquillage']

In [137]:
print(df_prod.select('prix').describe())

shape: (9, 2)
┌────────────┬───────────┐
│ statistic  ┆ prix      │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 18040.0   │
│ null_count ┆ 0.0       │
│ mean       ┆ 6.885182  │
│ std        ┆ 10.374337 │
│ min        ┆ 0.16      │
│ 25%        ┆ 2.67      │
│ 50%        ┆ 4.3       │
│ 75%        ┆ 8.5       │
│ max        ┆ 798.0     │
└────────────┴───────────┘


In [138]:
df_prod.filter(pl.col('prix') == 798)

ean,categorie,rayon,libelle_produit,prix
i64,str,str,str,f64
7148304214354,"""Boissons""","""champagnes""","""6l chp bollinger cuv.bt math+c""",798.0


### 3. Table client

In [139]:
get_infos(df_client)

Nombre de lignes : 2297
Nombre de colonnes : 2
------
Nombre de lignes en double : 0
Pourcentage de valeurs manquantes : 0.00%
-----
shape: (5, 2)
┌───────────────────┬──────────────────┐
│ custumer_id       ┆ date_inscription │
│ ---               ┆ ---              │
│ str               ┆ date             │
╞═══════════════════╪══════════════════╡
│ CUST-2KYXXXW1NK7I ┆ 2023-07-17       │
│ CUST-NR43XRT2PXYG ┆ 2023-09-07       │
│ CUST-CH58P8PSVIYU ┆ 2021-08-24       │
│ CUST-CI7JQHW4TIYT ┆ 2021-12-22       │
│ CUST-3QHP3KL4NPP2 ┆ 2020-12-20       │
└───────────────────┴──────────────────┘


In [140]:
df_client.select('custumer_id').n_unique()

2297

In [141]:
df_client = df_client.rename({'custumer_id':'customer_id'})

In [142]:
print(df_client.select('date_inscription').min())
print(df_client.select('date_inscription').max())

shape: (1, 1)
┌──────────────────┐
│ date_inscription │
│ ---              │
│ date             │
╞══════════════════╡
│ 2020-01-01       │
└──────────────────┘
shape: (1, 1)
┌──────────────────┐
│ date_inscription │
│ ---              │
│ date             │
╞══════════════════╡
│ 2024-08-14       │
└──────────────────┘


### 4. Table calendrier

In [152]:
get_infos(df_calendrier)

Nombre de lignes : 1999
Nombre de colonnes : 8
------
Nombre de lignes en double : 0
Pourcentage de valeurs manquantes : 0.00%
-----
shape: (5, 8)
┌────────────┬───────┬──────┬──────┬──────────┬────────────┬──────────────┬───────────┐
│ date       ┆ annee ┆ mois ┆ jour ┆ mois_nom ┆ annee_mois ┆ jour_semaine ┆ trimestre │
│ ---        ┆ ---   ┆ ---  ┆ ---  ┆ ---      ┆ ---        ┆ ---          ┆ ---       │
│ str        ┆ i64   ┆ i64  ┆ i8   ┆ str      ┆ str        ┆ i64          ┆ str       │
╞════════════╪═══════╪══════╪══════╪══════════╪════════════╪══════════════╪═══════════╡
│ 2020-01-01 ┆ 2020  ┆ 1    ┆ 1    ┆ janvier  ┆ 2020-01    ┆ 4            ┆ Q1        │
│ 2020-01-02 ┆ 2020  ┆ 1    ┆ 2    ┆ janvier  ┆ 2020-01    ┆ 5            ┆ Q1        │
│ 2020-01-03 ┆ 2020  ┆ 1    ┆ 3    ┆ janvier  ┆ 2020-01    ┆ 6            ┆ Q1        │
│ 2020-01-04 ┆ 2020  ┆ 1    ┆ 4    ┆ janvier  ┆ 2020-01    ┆ 7            ┆ Q1        │
│ 2020-01-05 ┆ 2020  ┆ 1    ┆ 5    ┆ janvier  ┆ 2020-01    ┆ 

In [144]:
df_calendrier = (
    df_calendrier.with_columns(
        pl.col('annee_mois')
        .map_elements(lambda x: date_serial_number(int(x)).strftime("%Y-%m"), return_dtype=pl.Utf8)
        .alias('annee_mois')
    )
)

In [145]:
df_calendrier = (
    df_calendrier.with_columns(
        pl.col('date')
        .map_elements(lambda x: date_serial_number(int(x)).strftime("%Y-%m-%d"), return_dtype=pl.Utf8)
        .alias('date')
    )
)

In [146]:
df_calendrier.select([pl.col(column).n_unique() for column in df_calendrier.columns])

date,annee,mois,jour,mois_nom,annee_mois,jour_semaine,trimestre
u32,u32,u32,u32,u32,u32,u32,u32
1999,6,12,31,12,66,7,4


In [147]:
df_calendrier = df_calendrier.with_columns(pl.col('jour').dt.day().alias('jour'))

### 5. Table employe

In [153]:
get_infos(df_employe)

Nombre de lignes : 56
Nombre de colonnes : 7
------
Nombre de lignes en double : 0
Pourcentage de valeurs manquantes : 0.00%
-----
shape: (5, 7)
┌────────────────┬───────────┬───────────┬──────────┬────────────┬────────────────┬────────────────┐
│ id_employe     ┆ employe   ┆ prenom    ┆ nom      ┆ date_debut ┆ hash_mdp       ┆ mail           │
│ ---            ┆ ---       ┆ ---       ┆ ---      ┆ ---        ┆ ---            ┆ ---            │
│ str            ┆ str       ┆ str       ┆ str      ┆ str        ┆ str            ┆ str            │
╞════════════════╪═══════════╪═══════════╪══════════╪════════════╪════════════════╪════════════════╡
│ 6fa61d0ecae0b5 ┆ lmaret    ┆ Laure     ┆ Maret    ┆ 2020-05-10 ┆ 0373c45921fbaa ┆ lmaret@supersm │
│ 63fef18d36b203 ┆           ┆           ┆          ┆            ┆ 7530f34b39e71b ┆ artmarket.fr   │
│ 9c…            ┆           ┆           ┆          ┆            ┆ a9…            ┆                │
│ 37c6a856b2e142 ┆ cvérany   ┆ Christian ┆ Véra

In [149]:
df_employe.select([pl.col(column).n_unique() for column in df_employe.columns])

id_employe,employe,prenom,nom,date_debut,hash_mdp,mail
u32,u32,u32,u32,u32,u32,u32
56,56,51,53,54,56,56


In [150]:
df_employe = (
    df_employe.with_columns(
        pl.col('date_debut')
        .map_elements(lambda x: date_serial_number(int(x)).strftime("%Y-%m-%d"), return_dtype=pl.Utf8)
        .alias('date_debut')
    )
)

In [151]:
print(df_employe.null_count())

shape: (1, 7)
┌────────────┬─────────┬────────┬─────┬────────────┬──────────┬──────┐
│ id_employe ┆ employe ┆ prenom ┆ nom ┆ date_debut ┆ hash_mdp ┆ mail │
│ ---        ┆ ---     ┆ ---    ┆ --- ┆ ---        ┆ ---      ┆ ---  │
│ u32        ┆ u32     ┆ u32    ┆ u32 ┆ u32        ┆ u32      ┆ u32  │
╞════════════╪═════════╪════════╪═════╪════════════╪══════════╪══════╡
│ 0          ┆ 0       ┆ 0      ┆ 0   ┆ 0          ┆ 0        ┆ 0    │
└────────────┴─────────┴────────┴─────┴────────────┴──────────┴──────┘


In [155]:
df_vente.write_csv("./data/vente.csv")
df_prod.write_csv("./data/produit.csv")
df_client.write_csv("./data/client.csv")
df_calendrier.write_csv("./data/calendrier.csv")
df_employe.write_csv("./data/employe.csv")